# IGL-CA: inference pipeline
Coordination analysis inference using the OpenIE6 model. 

* Anton's [OpenIE6 fork](https://github.com/alexeyev/openie6)
* [OpenIE6 original repo](https://github.com/dair-iitd/openie6)
* [OpenIE6 original paper](https://aclanthology.org/2020.emnlp-main.306/)

Prepared by [Anton Alekseev](https://github.com/alexeyev) and [Anastasia Predelina](https://github.com/PredelinaAsya). Quite a bit of effort, tbh.

**NOTA BENE**: GPU environment should be enabled before running the code! If not possible, another code cell for CPU-only environment is available at the very end of the notebook.

## Getting the OpenIE6 code

In [1]:
# ! rmdir openie6

In [2]:
! git clone https://github.com/alexeyev/openie6.git

Cloning into 'openie6'...
remote: Enumerating objects: 1876, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 1876 (delta 36), reused 29 (delta 19), pack-reused 1820
Receiving objects: 100% (1876/1876), 8.52 MiB | 16.15 MiB/s, done.
Resolving deltas: 100% (383/383), done.


In [3]:
! mkdir openie6/models

## Downloading the model

Let's get the CA model trained by the paper authors. Re-uploaded it to my gdrive to reduce the downloading time.

In [4]:
! gdown 185DK6eWKT6j6Oes04OIVFwJOKri8EVXV

Downloading...
From: https://drive.google.com/uc?id=185DK6eWKT6j6Oes04OIVFwJOKri8EVXV
To: /content/conj_model.zip
100% 3.61G/3.61G [01:10<00:00, 51.5MB/s]


In [5]:
! unzip conj_model.zip

Archive:  conj_model.zip
   creating: conj_model/
  inflating: conj_model/epoch=28_eval_acc=0.854.ckpt  
   creating: conj_model/logs/
   creating: conj_model/logs/test/
  inflating: conj_model/logs/test.txt  
   creating: conj_model/logs/test/checkpoints/
  inflating: conj_model/logs/test/events.out.tfevents.1601263425.vsky017.5076.0  
  inflating: conj_model/logs/test/events.out.tfevents.1601263521.vsky017.5784.0  
  inflating: conj_model/logs/test/hparams.yaml  


In [6]:
! rm conj_model.zip*
! mv conj_model* openie6/models/

## Creating a text sample

In [7]:
! echo "I love Luciano Pavarotti and Jose Carreras ." >> sentences.txt
! cat sentences.txt

I love Luciano Pavarotti and Jose Carreras .


## Setting up Python3.6

In [8]:
! sudo apt-get update -y -qq
! sudo apt-get install python3.6 python3.6-distutils python3.6-dev -qq 

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 9.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package libpython3.6-minimal:amd64.
(Reading database ... 122541 files and directories currently installed.)
Preparing to unpack .../0-libpython3.6-minimal_3.6.15-1+focal3_amd64.deb ...
Unpacking libpython3.6-minimal:amd64 (3.6.15-1+focal3) ...
Selecting previously unselected package python3.6-minimal.
Preparing to unpack .../1-python3.6-minimal_3.6.15-1+focal3_amd64.deb ...
Unpacking python3.6-minimal (3.6.15-1+focal3) ...
Selecting previously unselected package libpython3.6-stdlib:amd64.
Preparing to unpack .../

In [9]:
! ls /usr/bin/python3*

/usr/bin/python3	    /usr/bin/python3.6m
/usr/bin/python3.10	    /usr/bin/python3.6m-config
/usr/bin/python3.10-config  /usr/bin/python3.8
/usr/bin/python3.6	    /usr/bin/python3.8-config
/usr/bin/python3.6-config   /usr/bin/python3-config


Here you will have to select `python3.6` manually

In [10]:
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.6 3
!sudo update-alternatives --config python3

update-alternatives: using /usr/bin/python3.6 to provide /usr/bin/python3 (python3) in auto mode
There are 3 choices for the alternative python3 (providing /usr/bin/python3).

  Selection    Path                 Priority   Status
------------------------------------------------------------
* 0            /usr/bin/python3.6    3         auto mode
  1            /usr/bin/python3.10   2         manual mode
  2            /usr/bin/python3.6    3         manual mode
  3            /usr/bin/python3.8    1         manual mode

Press <enter> to keep the current choice[*], or type selection number: *
There are 3 choices for the alternative python3 (providing /usr/bin/python3).

  Selection    Path                 Priority   Status
------------------------------------------------------------
* 0            /usr/bin/python3.6    3         auto mode
  1            /usr/bin/python3.10   2         manual mode
  2            /usr/bin/python3.6    3         manual mode
  3            /usr/bin/python3.

## Setting up the dependencies

In [11]:
! rm get-pip*

rm: cannot remove 'get-pip*': No such file or directory


In [12]:
! python3 --version
! wget https://bootstrap.pypa.io/pip/3.6/get-pip.py
! python get-pip.py

Python 3.6.15
--2023-06-12 12:38:44--  https://bootstrap.pypa.io/pip/3.6/get-pip.py
Resolving bootstrap.pypa.io (bootstrap.pypa.io)... 151.101.0.175, 151.101.64.175, 151.101.128.175, ...
Connecting to bootstrap.pypa.io (bootstrap.pypa.io)|151.101.0.175|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2159061 (2.1M) [text/x-python]
Saving to: ‘get-pip.py’

get-pip.py          100%[===================>]   2.06M  --.-KB/s    in 0.08s   

2023-06-12 12:38:44 (26.6 MB/s) - ‘get-pip.py’ saved [2159061/2159061]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 4.3 MB/s            
     |████████████████████████████████| 952 kB 46.0 MB/s            


Satisfying a few more dependencies just in case

In [13]:
! python -m pip install -q wheel setuptools pip --upgrade

In [14]:
! python -m pip install -q -r openie6/requirements.txt

     |████████████████████████████████| 12.0 MB 3.9 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 776.8 MB 12 kB/s              
     |████████████████████████████████| 123 kB 52.3 MB/s            
     |████████████████████████████████| 104 kB 60.6 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 3.7 MB 40.8 MB/s            
     |████████████████████████████████| 156 kB 51.3 MB/s            
     |████████████████████████████████| 133 kB 81.0 MB/s            
     |████████████████████████████████| 82 kB 914 kB/s             
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 829 kB 49.0 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 90 kB 8.0 MB/s             
     |████████████████████████████████| 3.0 MB 42.4 MB/s            
     |████████████████████████████████| 58 kB 5.2 MB/s             
     |███

In [15]:
! python -m nltk.downloader stopwords
! python -m nltk.downloader punkt 

/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Colab-specific dependencies

In [16]:
! python -m pip install -q ipykernel google-colab ipywidgets

     |████████████████████████████████| 121 kB 4.1 MB/s            
     |████████████████████████████████| 72 kB 1.2 MB/s             
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 123 kB 16.4 MB/s            
     |████████████████████████████████| 130 kB 47.0 MB/s            
     |████████████████████████████████| 427 kB 20.1 MB/s            
     |████████████████████████████████| 64 kB 2.3 MB/s             
     |████████████████████████████████| 104 kB 47.3 MB/s            
     |████████████████████████████████| 758 kB 43.9 MB/s            
     |████████████████████████████████| 8.0 MB 48.2 MB/s            
     |████████████████████████████████| 10.1 MB 47.7 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 57 kB 3.5 MB/s             
     |████████████████████████████████| 484 kB 44.8 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 246 kB 46.4 

## Running coordination analysis model

Please don't pay attention to the `UnknownBackend` warnings. 

The `sentences.txt` samples will be converted to shorter sentences in `predictions.txt.conj`.

### GPU version (20 seconds or so)

In [ ]:
! cd openie6 && CUDA_DEVICE_ORDER=PCI_BUS_ID CUDA_VISIBLE_DEVICES=0 PYTHONPATH=imojie:imojie/allennlp:imojie/pytorch_transformers:$PYTHONPATH python run.py --save models/conj_model --mode predict --inp ../sentences.txt --batch_size 1 --model_str bert-large-cased --task conj --gpus 1 --out ../predictions.txt

Finally, the results.

In [ ]:
! cat predictions.txt.conj

### CPU version (40 seconds or so)

In [19]:
! cd openie6 && PYTHONPATH=imojie:imojie/allennlp:imojie/pytorch_transformers:$PYTHONPATH python run.py --save models/conj_model --mode predict --inp ../sentences.txt --batch_size 1 --model_str bert-large-cased --task conj --out ../predictions.txt

/usr/local/lib/python3.6/dist-packages/traitlets/config/configurable.py:84: UserWarning: Config option `use_jedi` not recognized by `IPCompleter`.
  self.config = config
[TerminalIPythonApp] WARNING | GUI event loop or pylab initialization failed

UnknownBackendTraceback (most recent call last)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py in enable_matplotlib(self, gui)
   2953         # Now we must activate the gui pylab wants to use, and fix %run to take
   2954         # plot updates into account
-> 2955         self.enable_gui(gui)
   2956         self.magics_manager.registry['ExecutionMagics'].default_runner = \
   2957             pt.mpl_runner(self.safe_execfile)

/usr/local/lib/python3.6/dist-packages/IPython/terminal/interactiveshell.py in enable_gui(self, gui)
    512         if gui:
    513             self.active_eventloop, self._inputhook =\
--> 514                 get_inputhook_name_and_func(gui)
    515         else:
    516             self.ac

In [20]:
! cat predictions.txt.conj

I love Luciano Pavarotti and Jose Carreras .
I love Luciano Pavarotti .
I love Jose Carreras .

